In [1]:
%matplotlib inline
import matplotlib.pyplot as plt # for plotting

import numpy as np
import pulp
from pulp import *

import scipy # various algorithms
import scipy.optimize
import scipy.interpolate
import scipy.signal
import random
import pandas as pd
import scipy as sp

import time

**Modeling MS2 fluorescence traces**

Data are fluorescence over time F(t) that we assume is being generated by turning a promoter on and off. When the promoter is on it generates transcriptions at a fixed rate R and that each transcript generates a fixed fluorescence intensity I for a fixed pulse interval (P) determined by the speed of the polymerase and the length of the transcript.

Thus F(t) = Sum (over t) of RI S(t) B(t) 
S(t) is a state vector that is 0 (off) or 1(on)
B(t) is a pulse vector that is 0 for times less than t, 1 between t and t+P, then 0 for times t+P

Everything in this model is fixed except for the state vector S(t), which we will fit using linear integer programming.

F(t) is loaded as a vector 
RI and P were determined by manually examining data. Can estimate RI by looking at the slope of traces that are increasing continuously. Get a value of ~25000. The pulse duration can be estimated from several observations. First, many traces reach a fluorescence intensity of 400,000. The maximum value you can achieve is the pulse RI * P. So if RI is 25,000 then P must be at least 16. This is consistent with the width of low-intensity pulses which probably correspond to a pulse furation less than the 6s time interval between. 

In [4]:
# Load data 

df = pd.read_csv('C:/Users/Nicholas/OneDrive/Berkeley Biophysics/projects/all_the_stripes/data/all_traces.csv', header=None)
#print(pd.DataFrame.head(df))

In [9]:
rows = [i for i in range(0,len(df))]
rows = [21]

fig = plt.figure(figsize=(8,4*len(rows)))
subplot = 0

#determined empirically

pulse_duration = 16
pulse_intensity = 25000

for d in rows:
    
    #data = np.nan_to_num(np.array(df.loc[d][1:]))  # first column is AP position - don't use for modeling
    data = np.array(df.iloc[d, :])
    datalen = len(data)
    
    # just for convenience later
    data_indices = [i for i in range(0,len(data))]

    # create basis vectors corresponding to pulses
    
    basis_vectors = []
    for x in range(0,datalen):
        basis_vector = datalen * [0]
        for i in range(x,min(x+pulse_duration,datalen)):
            basis_vector[i] = pulse_intensity    
        basis_vectors.append(np.array(basis_vector))
        
    # for convenience
    basis_indices = [i for i in range(len(basis_vectors))]
    
    # create variable for PuLP
    
    # basis loadings is 0 or 1 corresponding to if basis vector is used
    basis_loadings = [LpVariable("Basis vector " + str(b),0,1,cat="Integer") for b in basis_indices]
    
    # variable to capture to error in estimates
    err = [LpVariable("err " + str(i),0,None,cat="Continuous") for i in data_indices]
    
    # create a PuLP problem - going to minimize objective function
    prob = LpProblem("Fit MS2 Pulses",LpMinimize)

    # objective function is sum of errors
    prob += sum([err[i] for i in data_indices])
    
    # constraint is that modeled data less than error and greater than -error
    for i in data_indices:
        prob += data[i] - sum([basis_loadings[j] * basis_vectors[j][i] for j in basis_indices]) <= err[i]
        prob += data[i] - sum([basis_loadings[j] * basis_vectors[j][i] for j in basis_indices]) >= -err[i]
    
    # The problem data is written to an .lp file
    prob.writeLP("BasisTest.lp")
    
    # Solve problem
    prob.solve(pulp.GLPK())
    #prob.solve()
    
    # Calculate fit data
    
    fitdata = datalen * [0]
    fitdata = np.array(fitdata)
    basis_loading_vector = []

    for b in basis_indices:
        if basis_loadings[b].varValue == 1.0:
            fitdata += basis_vectors[b]
        basis_loading_vector.append(basis_loadings[b].varValue)
    # Plot
            
    subplot += 1
    ax = plt.subplot(len(rows),1,subplot)
    ax.plot(data,alpha=0.4)
    ax.plot(fitdata,color='r')
    ax.set_title("Trace " + str(d) + " with pulse intensity " + str(pulse_intensity) + " and pulse duration " + str(pulse_duration))
    ax2 = ax.twinx()
    ax2.bar(np.arange(0,len(basis_loading_vector)),basis_loading_vector, color='g',edgecolor = "none",alpha = 0.05)
    
plt.tight_layout()
plt.show()

TypeError: exceptions must be old-style classes or derived from BaseException, not str

In [3]:
# this will fit a pulse sequence from data
# if smooth is False it will just fit sequence
# if smooth is True it will apply smoothing, but runs slowly

def get_pulse_sequence(data,smooth=False,pulse_intensity=25000,pulse_duration=16,smoothp=100000):

    data = np.nan_to_num(data)
    datalen = len(data)
    data_indices = [i for i in range(0,datalen)]
    
    basis_vectors = []
    for x in range(0,datalen):
        basis_vector = datalen * [0]
        for i in range(x,min(x+pulse_duration,datalen)):
            basis_vector[i] = pulse_intensity 
        basis_vectors.append(np.array(basis_vector))
    
    basis_indices = [i for i in range(len(basis_vectors))]
    
    basis_loadings = [LpVariable("Basis vector " + str(b),0,1,cat="Integer") for b in basis_indices]
    err = [LpVariable("err " + str(i),0,None,cat="Continuous") for i in data_indices]
    
    if smooth == True:
        loadshifts = []
        for b in basis_indices[0:-1]:
            loadshifts.append(LpVariable("Loadshift " + str(b),0,1,cat="Continuous"))
            
    prob = LpProblem("Fit MS2 Pulses",LpMinimize)   
    
    if smooth == True:
        prob += sum([err[i] for i in data_indices]) + smoothp * sum([loadshifts[b] for b in basis_indices[0:-1]])
    else:
        prob += sum([err[i] for i in data_indices]) 

    for i in data_indices:
        prob += data[i] - sum([basis_loadings[j] * basis_vectors[j][i] for j in basis_indices]) <= err[i]
        prob += data[i] - sum([basis_loadings[j] * basis_vectors[j][i] for j in basis_indices]) >= -err[i]
    
    if smooth == True:
        for b in basis_indices[0:-1]:
            prob += basis_loadings[b+1]-basis_loadings[b] <= loadshifts[b]
            prob += basis_loadings[b+1]-basis_loadings[b] >= -loadshifts[b]
    
    prob.writeLP("MS2Fit.lp")
    prob.solve()
    
    fitdata = datalen * [0]
    fitdata = np.array(fitdata)

    basis_loading_vector = np.zeros(len(data))
    
    for b in basis_indices:
        if basis_loadings[b].varValue == 1.0:
            fitdata += basis_vectors[b]
            basis_loading_vector[b] = 1
    
    return (basis_loading_vector, fitdata)

In [4]:
# to work around slowness of smoothing
# this function fixes all of the pulses except in a specified window
# refits in that window only using smoothing penalty

def refine_pulse_sequence(data,pulse_sequence,win_start,win_end,pulse_intensity=25000,pulse_duration=16,smoothp=100000):

    data = np.nan_to_num(data)
    datalen = len(data)
    data_indices = [i for i in range(0,datalen)]
    
    basis_vectors = []
    for x in range(0,datalen):
        basis_vector = datalen * [0]
        for i in range(x,min(x+pulse_duration,datalen)):
            basis_vector[i] = pulse_intensity 
        basis_vectors.append(np.array(basis_vector))
    
    basis_indices = [i for i in range(len(basis_vectors))]
    
    basis_loadings = []
    
    for b in basis_indices:
        if b in range(win_start,win_end):
            basis_loadings.append(LpVariable("Basis vector " + str(b),0,1,cat="Integer"))
        else:
            basis_loadings.append(LpVariable("Basis vector " + str(b),pulse_sequence[b],pulse_sequence[b],cat="Integer"))
    
    err = [LpVariable("err " + str(i),0,None,cat="Continuous") for i in data_indices]
    
    loadshifts = (datalen-1) * [0]
    for b in range(win_start,win_end):
        loadshifts[b] = LpVariable("Loadshift " + str(b),0,1,cat="Continuous")
            
    prob = LpProblem("Fit MS2 Pulses",LpMinimize)   
    
    prob += sum([err[i] for i in data_indices]) + smoothp * sum([loadshifts[b] for b in range(win_start,win_end)])

    for i in data_indices:
        prob += data[i] - sum([basis_loadings[j] * basis_vectors[j][i] for j in basis_indices]) <= err[i]
        prob += data[i] - sum([basis_loadings[j] * basis_vectors[j][i] for j in basis_indices]) >= -err[i]
    
    for b in range(win_start,win_end-1):
        prob += basis_loadings[b+1]-basis_loadings[b] <= loadshifts[b]
        prob += basis_loadings[b+1]-basis_loadings[b] >= -loadshifts[b]
    
    prob.writeLP("MS2Fit.lp")
    prob.solve()
    
    fitdata = datalen * [0]
    fitdata = np.array(fitdata)

    basis_loading_vector = np.zeros(len(data))
    
    for b in basis_indices:
        if basis_loadings[b].varValue == 1.0:
            fitdata += basis_vectors[b]
            basis_loading_vector[b] = 1
    
    return (np.array(basis_loading_vector),fitdata)

In [7]:
#
# convenience function to compute data from pulse sequence
#

def get_data(pulse_sequence,pulse_intensity=25000,pulse_duration=16):
    
    datalen = len(pulse_sequence)
    data = datalen * [0]
    for i in range(0,datalen):
        if pulse_sequence[i] == 1:
            for j in range(i,min(datalen,i+pulse_duration)):
                data[j] += pulse_intensity
    
    return data

In [8]:
row = 8

data = np.nan_to_num(df.loc[row][0:])

# get unsmoothed pulse sequence

(pulse1,fit1) = get_pulse_sequence(data,False)

# copy to new pulse 

newpulse = np.copy(pulse1)

# smooth sequentially

# do first 40 in window of 60

(smoothpulse,smoothfit) = refine_pulse_sequence(data,newpulse,0,60)
newpulse[0:40] = smoothpulse[0:40]

# now set in windows of 20 with a buffer of 20

for i in range(40,len(data)-40,20):
    (smoothpulse,smoothfit) = refine_pulse_sequence(data,newpulse,i-20,i+40)
    newpulse[i:i+20] = smoothpulse[i:i+20]
    
# now do end - returned data are final

(newpulse,newdata) = refine_pulse_sequence(data,newpulse,len(data-60),len(data))

# plot

fig = plt.figure(figsize=(12,4))
ax = plt.subplot(1,1,1)
ax.plot(data,c='b',alpha=0.4)
ax.plot(fit1,c='r')
ax.plot(newdata,c='g')
plt.show()

AttributeError: 'NoneType' object has no attribute 'actualSolve'